In [1]:
import csv
import sys
import os
import re
import math
import random
import numpy as np
import numpy.linalg as npla
import scipy
import sklearn
from scipy import sparse
from scipy import linalg
import scipy.sparse.linalg as spla
from scipy.spatial import distance
import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib.mlab as mlab
from mpl_toolkits.mplot3d import axes3d
import operator
import statistics
import networkx as nx
import matplotlib.pyplot as plt
from multiprocessing.pool import ThreadPool


In [2]:
def generate_edges(writer, filename, update_canvas):
    seg = np.load(filename)
    for x in range(0, 1001):
        for y in range(0, 1001):
            curr_pixel_seg_num = seg[y,x]
            if update_canvas[y,x,0]=="0.0":
                continue
            for tup in [(0,1),(1,0)]:
                i = tup[0]
                j = tup[1]
                if x + i < 0 or x + i >= 1001 or y + j < 0 or y + j >= 1001:
                    continue
                if update_canvas[y+j,x+i,0] == "0.0": 
                    continue
#                 print("i,j", i,j)
#                 print("tuple", tup)
#                 print("x,y", x, y)
#                 print("x+i, y+j", x+i,y+j)
                seg_num = seg[y+j,x+i]
#                 print("seg_num",seg_num)
#                 print("curr_seg_num", curr_pixel_seg_num)
#                 print(seg)
                if seg_num == curr_pixel_seg_num:
                    size = np.count_nonzero(seg == seg_num)
                    up1 = update_canvas[y,x]
                    up1_time = int(up1[1])
                    up2 = update_canvas[y+j,x+i]
                    up2_time = int(up2[1])
                    if up1_time < up2_time:
                        writer.writerow([up1[0], up2[0], str(size)])
                    else:
                        writer.writerow([up2[0], up1[0], str(size)])

In [3]:
update_canvas = np.array(np.zeros((1001,1001,6)), dtype = 'str')
input_file = "../data/sorted_tile_placements_idx.csv"
curr_time = 1490918688000
with open(input_file, 'r') as file:
    next(file, None)
    reader = csv.reader(file)
    for r in reader:
        x = int(r[3])
        y = int(r[4])
        time = int(r[1])
        idx = int(r[0])
        if curr_time != time:
            filename = "/home/yw180/place/data/seg_out_last_20/"+str(time) +".npy"
            if os.path.exists(filename):
     #           print("in exists")
                with open("/home/yw180/place/data/edge_list_last_20/"+str(time) +".csv", 'w') as file_out:
                    writer = csv.writer(file_out, delimiter = ",")
                    generate_edges(writer, filename, update_canvas)
            curr_time = time
        update_canvas[y,x] = r